# Replication Notebook for Simulations: Sample Fit Reliability

Gabriel Okasa and Kenneth A. Younge

In [ ]:
# get current working directory
path = %pwd

## Libraries

In [ ]:
# Python version 3.8.8
import statsmodels # version 0.12.2

import pandas as pd # version 1.3.5
import numpy as np # version 1.22.0
import samplefit as sf # version 0.3.1
import statsmodels.api as sm
import matplotlib.pyplot as plt # version 3.4.2

from scipy import stats # version 1.7.2
from sklearn.linear_model import RANSACRegressor # version 1.1.1
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

# turn off future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# panda decimal setting
pd.set_option('display.float_format', '{:.4f}'.format)

## DGPs

In [ ]:
# define DGP function
def dgp(n_obs=1000, n_vars=1, beta=1, intercept=False, outlier_share=0.00, leverage=True, symmetry=False, verbose=True, path=None, seed=0):
    """
    Data Generating Process.

    Parameters
    ----------
    n_obs : TYPE: int
        DESCRIPTION: number of observations to generate. Default is 1000.
    n_vars : TYPE: int
        DESCRIPTION: number of variables to generate. Default is 1.
    beta : TYPE: float
        DESCRIPTION: effect size of covariates. Default is 1.
    intercept : TYPE: bool
        DESCRIPTION: should intercept be included? Default is False.
    outlier_share : TYPE: float
        DESCRIPTION: share of outliers in the sample. Default is 0.00, no outliers.
    leverage : TYPE: bool
        DESCRIPTION: should outliers have leverage? Default is True.
    symmetry : TYPE: bool
        DESCRIPTION: should outliers be split into symmetric clusters? Default is False.
    verbose : TYPE: bool
        DESCRIPTION: should data plots be printed? Default is True.
    path : TYPE: str
        DESCRIPTION: path where the figure should be saved? Default is None.
    seed : TYPE: int
        DESCRIPTION: random seed for replicability. Default is 0.

    Returns
    -------
    pd.DataFrame with the simulated data.
    """
    
    # general definitions
    # set random seed for replicability
    np.random.seed(seed)
    
    # determine number of inliers vs outliers
    n_outliers = int(np.round(outlier_share * n_obs))
    n_inliers = n_obs - n_outliers
    
    ## simulate inliers
    # set beta according to the number of variables
    beta = pd.Series(np.repeat(beta, n_vars))
    # simulate feature x from standard normal distribution
    x_var = pd.DataFrame(np.random.randn(n_inliers*n_vars).reshape((n_inliers, n_vars)),
                         columns=[('x'+str(varidx)) for varidx in range(n_vars)])
    # check if intercept should be included
    if intercept:
        # prepend column of ones
        x_var = pd.concat([pd.Series(np.ones(len(x_var)), index=x_var.index, name='intercept'), x_var], axis=1)
        # prepend beta zero coefficient for intercept
        beta = pd.concat([pd.Series(1), beta])
    # get the matrix product x times beta
    x_beta = np.dot(beta, x_var.T)
    # simulate the outcome from the gaussian distribution
    y_var = pd.Series(np.random.normal(x_beta, 2/3), name='y')
    #y_var = pd.Series(np.random.normal(x_beta, 1), name='y')
    # pack the data into a dataframe
    data_in = pd.DataFrame(pd.concat([y_var, x_var], axis=1))

    if n_outliers > 0:
        # simulate feature x from standard normal distribution
        x_out = pd.DataFrame(np.random.normal(0, 1/2, n_outliers*n_vars).reshape((n_outliers, n_vars)), columns=[('x0')])
        if intercept:
            # prepend column of ones
            x_out = pd.concat([pd.Series(np.ones(x_out.shape[0]), index=x_out.index, name='intercept'), x_out], axis=1)
        # check if leverage
        if leverage:
            shift = 2
        else:
            shift = 0
        # chekc if symmetry
        if symmetry:
            # divide outliers into 2 groups 50:50
            obs_out_up = np.random.choice(x_out.index, size=round(0.5*x_out.shape[0]), replace=False)
            obs_out_down = np.setdiff1d(x_out.index, obs_out_up)
            # shift values of x
            x_out.loc[obs_out_up, x_out.columns != 'intercept'] = x_out.loc[obs_out_up, x_out.columns != 'intercept'] - shift # shift X to create leverage
            x_out.loc[obs_out_down, x_out.columns != 'intercept'] = x_out.loc[obs_out_down, x_out.columns != 'intercept'] + shift # shift X to create leverage
            # outcome with different phenomenon (constant outcome without dependence on X)
            y_out = pd.Series(np.random.normal(0, 1, x_out.shape[0]), name='y')
            y_out[obs_out_up] = pd.Series(np.random.normal(1, 2/3, len(obs_out_up)), name='y')
            y_out[obs_out_down] = pd.Series(np.random.normal(-1, 2/3, len(obs_out_down)), name='y')
        elif not symmetry and not leverage:
            x_out = pd.DataFrame(np.random.normal(0, 2, n_outliers*n_vars).reshape((n_outliers, n_vars)), columns=[('x0')])
            if intercept:
                # prepend column of ones
                x_out = pd.concat([pd.Series(np.ones(x_out.shape[0]), index=x_out.index, name='intercept'), x_out], axis=1)
            # outcome with different phenomenon (constant outcome without dependence on X)
            y_out = pd.Series(np.random.normal(0, 2, x_out.shape[0]), name='y')
        else:
            # shift values
            x_out.loc[:, x_out.columns != 'intercept'] = x_out.loc[:, x_out.columns != 'intercept'] + shift # shift X to create leverage
            # outcome with different phenomenon (constant outcome without dependence on X)
            y_out = pd.Series(np.random.normal(-1, 2/3, x_out.shape[0]), name='y')
        # pack data into dataframe
        data_out = pd.DataFrame(pd.concat([y_out, x_out], axis=1))
    else:
        data_out = None
    
    # combine in and outliers
    if data_out is None:
        data = data_in.copy()
    else:
        data = pd.DataFrame(pd.concat([data_in, data_out])).reset_index()
    
    # check the distribution of the treatment and the outcome
    if verbose:
        ax = data.loc[:, data.columns != 'intercept'].hist(layout=(1, data.shape[1]), color='grey', alpha=0.5, figsize = (7.5, 5))
        fig = ax[0][0].get_figure()
        if path is not None:
            fig.savefig((path+'/descriptives_' + str(outlier_share) + str(leverage) + str(symmetry) + '.png'), bbox_inches='tight')
    
    # return results
    return data

## Simulation

### Scenario 1: No Outliers

In [ ]:
# simulate data
true_slope=1
data = dgp(beta=true_slope, outlier_share=0.00, verbose=False, seed=0)

# graphical evidence
# specify model
model = sm.OLS(endog=data.y, exog=pd.DataFrame(sm.add_constant(data.x0)))

# fit ols
ols_fit = model.fit()
ols_pred = ols_fit.fittedvalues
ols_weights = np.ones(len(data.y))

# fit huber
huber_fit = sm.RLM(endog=data.y, exog=pd.DataFrame(sm.add_constant(data.x0)), M=sm.robust.norms.HuberT()).fit()
huber_pred = huber_fit.fittedvalues
huber_weights = huber_fit.weights

# fit ransac
ransac_fit = RANSACRegressor(random_state=0)
ransac_fit.fit(X=pd.DataFrame(data.x0), y=data.y)
ransac_pred = pd.Series(ransac_fit.predict(X=pd.DataFrame(data.x0)))
ransac_weights = ransac_fit.inlier_mask_*1

# fit sfr
sfr = sf.SFR(linear_model=model, random_state=0)
sfr_fit = sfr.fit()
sfr_pred = pd.Series(sfr_fit.fittedvalues)
sfr_scores = sfr.score()
sfr_weights = sfr_fit.weights

In [ ]:
# save results for later plots
data_sim1 = data.copy()
scores_sim1 = sfr_scores.scores.copy()

#### Annealing

In [ ]:
sfr_annealing = sfr.anneal(share=0.1, n_boot=1000)
sfr_annealing.plot(xname='x0', title='Scenario 1: No Outliers', xlabel='X', ylim=[0.6,1.2], dpi=300,
                   path=path+'/figures/', fname='sim1_annealing.png')

In [ ]:
# format the plot
sfr_annealing.figures['x0'][1].axhline(y=1, color='grey', linestyle='--', xmin=0, xmax=1.5, label="True")
labels = ["Estimate", "True", "95% CI"]
sfr_annealing.figures['x0'][1].get_legend().remove()
handles, _ = sfr_annealing.figures['x0'][1].get_legend_handles_labels()
# Slice list to remove first handle
sfr_annealing.figures['x0'][0].legend(handles = handles, labels = labels, loc = 'upper left', bbox_to_anchor=(0.125, 0.88))
sfr_annealing.figures['x0'][0]

In [ ]:
# save the plot
sfr_annealing.figures['x0'][0].savefig(path+'/figures/sim1_annealing.png')

#### Scoring

In [ ]:
# plot the scores
sfr_scores.plot(xname='x0', xlabel='X', yname='Y', figsize=(10,5), ylim=[-7.5,7.5], xlim=[-5,5], title='Scenario 1: No Outliers', dpi=300,
                path=path+'/figures', fname='sim1_scores.png')

#### Fitting

In [ ]:
# evaluate slope estimate

# bias
ols_bias_summary = {}
huber_bias_summary = {}
ransac_bias_summary = {}
sfr_bias_summary = {}

# mse
ols_mse_summary = {}
huber_mse_summary = {}
ransac_mse_summary = {}
sfr_mse_summary = {}

# SD
ols_sd_summary = {}
huber_sd_summary = {}
ransac_sd_summary = {}
sfr_sd_summary = {}

# JB
ols_jb_summary = {}
huber_jb_summary = {}
ransac_jb_summary = {}
sfr_jb_summary = {}

# true slope
true_slope = 1
sim_reps = 1000

# settings
obs_range = [100, 500, 1000]
outlier_share_range = [0.00]

# run simulation
for obs_idx in obs_range:
    for outlier_share_idx in outlier_share_range:
        
        # storage
        # bias
        ols_bias = []
        huber_bias = []
        ransac_bias = []
        sfr_bias = []
        
        # mse
        ols_mse = []
        huber_mse = []
        ransac_mse = []
        sfr_mse = []

        # storage for SD and JB
        ols_coef = []
        huber_coef = []
        ransac_coef = []
        sfr_coef = []
        
        for sim_idx in range(sim_reps):
            
            # generate data
            data = dgp(beta=true_slope, n_obs=obs_idx, outlier_share=outlier_share_idx, verbose=False, seed=sim_idx)

            # specify model
            model = sm.OLS(endog=data.y, exog=pd.DataFrame(sm.add_constant(data.x0)))

            # fit ols
            ols_fit = model.fit()
            ols_bias.append(np.abs(true_slope - ols_fit.params[1]))
            ols_mse.append((true_slope - ols_fit.params[1]) ** 2)
            ols_coef.append(ols_fit.params[1])

            # fit huber
            np.random.seed(sim_idx)
            huber_fit = sm.RLM(endog=data.y, exog=pd.DataFrame(sm.add_constant(data.x0)), M=sm.robust.norms.HuberT()).fit()
            huber_bias.append(np.abs(true_slope - huber_fit.params[1]))
            huber_mse.append((true_slope - huber_fit.params[1]) ** 2)
            huber_coef.append(huber_fit.params[1])

            # fit ransac
            ransac_fit = RANSACRegressor(random_state=sim_idx)
            ransac_fit.fit(X=pd.DataFrame(data.x0), y=data.y)
            ransac_bias.append(np.abs(true_slope - ransac_fit.estimator_.coef_[0]))
            ransac_mse.append((true_slope - ransac_fit.estimator_.coef_[0]) ** 2)
            ransac_coef.append(ransac_fit.estimator_.coef_[0])

            # fit sfr
            sfr_fit = sf.SFR(linear_model=model, random_state=sim_idx).fit()
            sfr_bias.append(np.abs(true_slope - sfr_fit.params[1]))
            sfr_mse.append((true_slope - sfr_fit.params[1]) ** 2)
            sfr_coef.append(sfr_fit.params[1])
        
        # summarize resurls
        # ols
        ols_bias_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(ols_bias)
        ols_mse_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(ols_mse)
        ols_sd_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.std(ols_coef)
        ols_jb_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = stats.jarque_bera(ols_coef).pvalue
        
        # huber
        huber_bias_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(huber_bias)
        huber_mse_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(huber_mse)
        huber_sd_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.std(huber_coef)
        huber_jb_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = stats.jarque_bera(huber_coef).pvalue
        
        # ransac
        ransac_bias_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(ransac_bias)
        ransac_mse_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(ransac_mse)
        ransac_sd_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.std(ransac_coef)
        ransac_jb_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = stats.jarque_bera(ransac_coef).pvalue
        
        # sfr
        sfr_bias_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(sfr_bias)
        sfr_mse_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(sfr_mse)
        sfr_sd_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.std(sfr_coef)
        sfr_jb_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = stats.jarque_bera(sfr_coef).pvalue

In [ ]:
# summarize results
# bias
sim1_bias_results = pd.concat([pd.DataFrame(ols_bias_summary, index=['OLS']), pd.DataFrame(huber_bias_summary, index=['HUBER']),
                               pd.DataFrame(ransac_bias_summary, index=['RANSAC']), pd.DataFrame(sfr_bias_summary, index=['SFR'])])
# mse
sim1_mse_results = pd.concat([pd.DataFrame(ols_mse_summary, index=['OLS']), pd.DataFrame(huber_mse_summary, index=['HUBER']),
                              pd.DataFrame(ransac_mse_summary, index=['RANSAC']), pd.DataFrame(sfr_mse_summary, index=['SFR'])])
# sd
sim1_sd_results = pd.concat([pd.DataFrame(ols_sd_summary, index=['OLS']), pd.DataFrame(huber_sd_summary, index=['HUBER']),
                              pd.DataFrame(ransac_sd_summary, index=['RANSAC']), pd.DataFrame(sfr_sd_summary, index=['SFR'])])
# jb
sim1_jb_results = pd.concat([pd.DataFrame(ols_jb_summary, index=['OLS']), pd.DataFrame(huber_jb_summary, index=['HUBER']),
                              pd.DataFrame(ransac_jb_summary, index=['RANSAC']), pd.DataFrame(sfr_jb_summary, index=['SFR'])])

In [ ]:
# get latex
sim1_all_results = pd.concat([sim1_mse_results, sim1_bias_results, sim1_sd_results, sim1_jb_results])
print(sim1_all_results.to_latex(caption='Simulation Results for Effect Estimation - Scenario 1: No Outliers'))
# save
sim1_all_results.to_csv(path+'/results/sim1_results.csv', index=False)

### Scenario 2: Randomly Distributed Outliers

In [ ]:
# simulate data
data = dgp(beta=true_slope, outlier_share=0.05, leverage=False, verbose=False, seed=0)

# graphical evidence
# specify model
model = sm.OLS(endog=data.y, exog=pd.DataFrame(sm.add_constant(data.x0)))

# fit ols
ols_fit = model.fit()
ols_pred = ols_fit.fittedvalues
ols_weights = np.ones(len(data.y))

# fit huber
huber_fit = sm.RLM(endog=data.y, exog=pd.DataFrame(sm.add_constant(data.x0)), M=sm.robust.norms.HuberT()).fit()
huber_pred = huber_fit.fittedvalues
huber_weights = huber_fit.weights

# fit ransac
ransac_fit = RANSACRegressor(random_state=0)
ransac_fit.fit(X=pd.DataFrame(data.x0), y=data.y)
ransac_pred = pd.Series(ransac_fit.predict(X=pd.DataFrame(data.x0)))
ransac_weights = ransac_fit.inlier_mask_*1

# fit sfr
sfr = sf.SFR(linear_model=model, random_state=0)
sfr_fit = sfr.fit()
sfr_pred = pd.Series(sfr_fit.fittedvalues)
sfr_scores = sfr.score()
sfr_weights = sfr_fit.weights

In [ ]:
# save results for later plots
data_sim2 = data.copy()
scores_sim2 = sfr_scores.scores.copy()

#### Annealing

In [ ]:
sfr_annealing = sfr.anneal(share=0.1, n_boot=1000)
sfr_annealing.plot(xname='x0', title='Scenario 2: Randomly Distributed Outliers', dpi=300, ylim=[0.6,1.2],
                   path=path+'/figures', fname='sim2_annealing.png')

In [ ]:
# format
sfr_annealing.figures['x0'][1].axhline(y=1, color='grey', linestyle='--', xmin=0, xmax=1.5, label="True")
labels = ["Estimate", "True", "95% CI"]
sfr_annealing.figures['x0'][1].get_legend().remove()
handles, _ = sfr_annealing.figures['x0'][1].get_legend_handles_labels()
# Slice list to remove first handle
sfr_annealing.figures['x0'][0].legend(handles = handles, labels = labels, loc = 'upper left', bbox_to_anchor=(0.125, 0.88))
sfr_annealing.figures['x0'][0]

In [ ]:
# save
sfr_annealing.figures['x0'][0].savefig(path+'/figures/sim2_annealing.png')

#### Scoring

In [ ]:
# plot the scores
sfr_scores.plot(xname='x0', xlabel='X', yname='Y', figsize=(10,5), ylim=[-7.5,7.5], xlim=[-5,5], title='Scenario 2: Randomly Distributed Outliers', dpi=300,
                path=path+'/figures', fname='sim2_scores.png')

#### Fitting

In [ ]:
# evaluate slope estimate

# bias
ols_bias_summary = {}
huber_bias_summary = {}
ransac_bias_summary = {}
sfr_bias_summary = {}

# mse
ols_mse_summary = {}
huber_mse_summary = {}
ransac_mse_summary = {}
sfr_mse_summary = {}

# SD
ols_sd_summary = {}
huber_sd_summary = {}
ransac_sd_summary = {}
sfr_sd_summary = {}

# JB
ols_jb_summary = {}
huber_jb_summary = {}
ransac_jb_summary = {}
sfr_jb_summary = {}

# true slope
true_slope = 1
sim_reps = 1000

# settings
obs_range = [100, 500, 1000]
outlier_share_range = [0.01, 0.025, 0.05]

# run simulation
for obs_idx in obs_range:
    for outlier_share_idx in outlier_share_range:
        
        # storage
        # bias
        ols_bias = []
        huber_bias = []
        ransac_bias = []
        sfr_bias = []
        
        # mse
        ols_mse = []
        huber_mse = []
        ransac_mse = []
        sfr_mse = []

        # storage for SD and JB
        ols_coef = []
        huber_coef = []
        ransac_coef = []
        sfr_coef = []
        
        for sim_idx in range(sim_reps):
            
            # generate data
            data = dgp(beta=true_slope, n_obs=obs_idx, outlier_share=outlier_share_idx, leverage=False, symmetry=False, verbose=False, seed=sim_idx)

            # specify model
            model = sm.OLS(endog=data.y, exog=pd.DataFrame(sm.add_constant(data.x0)))

            # fit ols
            ols_fit = model.fit()
            ols_bias.append(np.abs(true_slope - ols_fit.params[1]))
            ols_mse.append((true_slope - ols_fit.params[1]) ** 2)
            ols_coef.append(ols_fit.params[1])

            # fit huber
            np.random.seed(sim_idx)
            huber_fit = sm.RLM(endog=data.y, exog=pd.DataFrame(sm.add_constant(data.x0)), M=sm.robust.norms.HuberT()).fit()
            huber_bias.append(np.abs(true_slope - huber_fit.params[1]))
            huber_mse.append((true_slope - huber_fit.params[1]) ** 2)
            huber_coef.append(huber_fit.params[1])

            # fit ransac
            ransac_fit = RANSACRegressor(random_state=sim_idx)
            ransac_fit.fit(X=pd.DataFrame(data.x0), y=data.y)
            ransac_bias.append(np.abs(true_slope - ransac_fit.estimator_.coef_[0]))
            ransac_mse.append((true_slope - ransac_fit.estimator_.coef_[0]) ** 2)
            ransac_coef.append(ransac_fit.estimator_.coef_[0])

            # fit sfr
            sfr_fit = sf.SFR(linear_model=model, random_state=sim_idx).fit()
            sfr_bias.append(np.abs(true_slope - sfr_fit.params[1]))
            sfr_mse.append((true_slope - sfr_fit.params[1]) ** 2)
            sfr_coef.append(sfr_fit.params[1])
        
        # summarize resurls
        # ols
        ols_bias_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(ols_bias)
        ols_mse_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(ols_mse)
        ols_sd_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.std(ols_coef)
        ols_jb_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = stats.jarque_bera(ols_coef).pvalue
        
        # huber
        huber_bias_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(huber_bias)
        huber_mse_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(huber_mse)
        huber_sd_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.std(huber_coef)
        huber_jb_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = stats.jarque_bera(huber_coef).pvalue
        
        # ransac
        ransac_bias_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(ransac_bias)
        ransac_mse_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(ransac_mse)
        ransac_sd_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.std(ransac_coef)
        ransac_jb_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = stats.jarque_bera(ransac_coef).pvalue
        
        # sfr
        sfr_bias_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(sfr_bias)
        sfr_mse_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(sfr_mse)
        sfr_sd_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.std(sfr_coef)
        sfr_jb_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = stats.jarque_bera(sfr_coef).pvalue

In [ ]:
# summarize results
# bias
sim2_bias_results = pd.concat([pd.DataFrame(ols_bias_summary, index=['OLS']), pd.DataFrame(huber_bias_summary, index=['HUBER']),
                               pd.DataFrame(ransac_bias_summary, index=['RANSAC']), pd.DataFrame(sfr_bias_summary, index=['SFR'])])
# mse
sim2_mse_results = pd.concat([pd.DataFrame(ols_mse_summary, index=['OLS']), pd.DataFrame(huber_mse_summary, index=['HUBER']),
                              pd.DataFrame(ransac_mse_summary, index=['RANSAC']), pd.DataFrame(sfr_mse_summary, index=['SFR'])])
# sd
sim2_sd_results = pd.concat([pd.DataFrame(ols_sd_summary, index=['OLS']), pd.DataFrame(huber_sd_summary, index=['HUBER']),
                              pd.DataFrame(ransac_sd_summary, index=['RANSAC']), pd.DataFrame(sfr_sd_summary, index=['SFR'])])
# jb
sim2_jb_results = pd.concat([pd.DataFrame(ols_jb_summary, index=['OLS']), pd.DataFrame(huber_jb_summary, index=['HUBER']),
                              pd.DataFrame(ransac_jb_summary, index=['RANSAC']), pd.DataFrame(sfr_jb_summary, index=['SFR'])])

In [ ]:
# get latex
sim2_all_results = pd.concat([sim2_mse_results, sim2_bias_results, sim2_sd_results, sim2_jb_results])
print(sim2_all_results.to_latex(caption='Simulation Results for Effect Estimation - Scenario 2: Randomly Distributed Outliers'))
# save
sim2_all_results.to_csv(path+'/results/sim2_results.csv', index=False)

### Scenario 3: Densely Distributed Outliers with Leverage

In [ ]:
# simulate data
true_slope=1
data = dgp(beta=true_slope, outlier_share=0.05, leverage=True, verbose=False, seed=0)

# graphical evidence
# specify model
model = sm.OLS(endog=data.y, exog=pd.DataFrame(sm.add_constant(data.x0)))

# fit ols
ols_fit = model.fit()
ols_pred = ols_fit.fittedvalues
ols_weights = np.ones(len(data.y))

# fit huber
huber_fit = sm.RLM(endog=data.y, exog=pd.DataFrame(sm.add_constant(data.x0)), M=sm.robust.norms.HuberT()).fit()
huber_pred = huber_fit.fittedvalues
huber_weights = huber_fit.weights

# fit ransac
ransac_fit = RANSACRegressor(random_state=0)
ransac_fit.fit(X=pd.DataFrame(data.x0), y=data.y)
ransac_pred = pd.Series(ransac_fit.predict(X=pd.DataFrame(data.x0)))
ransac_weights = ransac_fit.inlier_mask_*1

# fit sfr
sfr = sf.SFR(linear_model=model, random_state=0)
sfr_fit = sfr.fit()
sfr_pred = pd.Series(sfr_fit.fittedvalues)
sfr_scores = sfr.score()
sfr_weights = sfr_fit.weights

In [ ]:
# save results for later plots
data_sim3 = data.copy()
scores_sim3 = sfr_scores.scores.copy()

#### Annealing

In [ ]:
sfr_annealing = sfr.anneal(share=0.1, n_boot=1000)
sfr_annealing.plot(xname='x0', title='Scenario 3: Densely Distributed Outliers with Leverage', ylim=[0.6,1.2], dpi=300,
                   path=path+'/figures', fname='sim3_annealing.png')

In [ ]:
# format
sfr_annealing.figures['x0'][1].axhline(y=1, color='grey', linestyle='--', xmin=0, xmax=1.5, label="True")
labels = ["Estimate", "True", "95% CI"]
sfr_annealing.figures['x0'][1].get_legend().remove()
handles, _ = sfr_annealing.figures['x0'][1].get_legend_handles_labels()
# Slice list to remove first handle
sfr_annealing.figures['x0'][0].legend(handles = handles, labels = labels, loc = 'upper left', bbox_to_anchor=(0.125, 0.88))
sfr_annealing.figures['x0'][0]

In [ ]:
# save
sfr_annealing.figures['x0'][0].savefig(path+'/figures/sim3_annealing.png')

#### Scoring

In [ ]:
# plot the scores
sfr_scores.plot(xname='x0', xlabel='X', yname='Y', figsize=(10,5), ylim=[-7.5,7.5], xlim=[-5,5], title='Scenario 3: Densely Distributed Outliers with Leverage',
                dpi=300, path=path+'/figures', fname='sim3_scores.png')

#### Fitting

In [ ]:
# evaluate slope estimate

# bias
ols_bias_summary = {}
huber_bias_summary = {}
ransac_bias_summary = {}
sfr_bias_summary = {}

# mse
ols_mse_summary = {}
huber_mse_summary = {}
ransac_mse_summary = {}
sfr_mse_summary = {}

# SD
ols_sd_summary = {}
huber_sd_summary = {}
ransac_sd_summary = {}
sfr_sd_summary = {}

# JB
ols_jb_summary = {}
huber_jb_summary = {}
ransac_jb_summary = {}
sfr_jb_summary = {}

# true slope
true_slope = 1
sim_reps = 1000

# settings
obs_range = [100, 500, 1000]
outlier_share_range = [0.01, 0.025, 0.05]

# run simulation
for obs_idx in obs_range:
    for outlier_share_idx in outlier_share_range:
        
        # storage
        # bias
        ols_bias = []
        huber_bias = []
        ransac_bias = []
        sfr_bias = []
        
        # mse
        ols_mse = []
        huber_mse = []
        ransac_mse = []
        sfr_mse = []

        # storage for SD and JB
        ols_coef = []
        huber_coef = []
        ransac_coef = []
        sfr_coef = []
        
        for sim_idx in range(sim_reps):
            
            # generate data
            data = dgp(beta=true_slope, n_obs=obs_idx, outlier_share=outlier_share_idx, leverage=True, symmetry=False, verbose=False, seed=sim_idx)

            # specify model
            model = sm.OLS(endog=data.y, exog=pd.DataFrame(sm.add_constant(data.x0)))

            # fit ols
            ols_fit = model.fit()
            ols_bias.append(np.abs(true_slope - ols_fit.params[1]))
            ols_mse.append((true_slope - ols_fit.params[1]) ** 2)
            ols_coef.append(ols_fit.params[1])

            # fit huber
            np.random.seed(sim_idx)
            huber_fit = sm.RLM(endog=data.y, exog=pd.DataFrame(sm.add_constant(data.x0)), M=sm.robust.norms.HuberT()).fit()
            huber_bias.append(np.abs(true_slope - huber_fit.params[1]))
            huber_mse.append((true_slope - huber_fit.params[1]) ** 2)
            huber_coef.append(huber_fit.params[1])

            # fit ransac
            ransac_fit = RANSACRegressor(random_state=sim_idx)
            ransac_fit.fit(X=pd.DataFrame(data.x0), y=data.y)
            ransac_bias.append(np.abs(true_slope - ransac_fit.estimator_.coef_[0]))
            ransac_mse.append((true_slope - ransac_fit.estimator_.coef_[0]) ** 2)
            ransac_coef.append(ransac_fit.estimator_.coef_[0])

            # fit sfr
            sfr_fit = sf.SFR(linear_model=model, random_state=sim_idx).fit()
            sfr_bias.append(np.abs(true_slope - sfr_fit.params[1]))
            sfr_mse.append((true_slope - sfr_fit.params[1]) ** 2)
            sfr_coef.append(sfr_fit.params[1])
        
        # summarize resurls
        # ols
        ols_bias_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(ols_bias)
        ols_mse_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(ols_mse)
        ols_sd_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.std(ols_coef)
        ols_jb_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = stats.jarque_bera(ols_coef).pvalue
        
        # huber
        huber_bias_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(huber_bias)
        huber_mse_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(huber_mse)
        huber_sd_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.std(huber_coef)
        huber_jb_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = stats.jarque_bera(huber_coef).pvalue
        
        # ransac
        ransac_bias_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(ransac_bias)
        ransac_mse_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(ransac_mse)
        ransac_sd_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.std(ransac_coef)
        ransac_jb_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = stats.jarque_bera(ransac_coef).pvalue
        
        # sf
        sfr_bias_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(sfr_bias)
        sfr_mse_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(sfr_mse)
        sfr_sd_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.std(sfr_coef)
        sfr_jb_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = stats.jarque_bera(sfr_coef).pvalue

In [ ]:
# summarize results
# bias
sim3_bias_results = pd.concat([pd.DataFrame(ols_bias_summary, index=['OLS']), pd.DataFrame(huber_bias_summary, index=['HUBER']),
                               pd.DataFrame(ransac_bias_summary, index=['RANSAC']), pd.DataFrame(sfr_bias_summary, index=['SFR'])])
# mse
sim3_mse_results = pd.concat([pd.DataFrame(ols_mse_summary, index=['OLS']), pd.DataFrame(huber_mse_summary, index=['HUBER']),
                              pd.DataFrame(ransac_mse_summary, index=['RANSAC']), pd.DataFrame(sfr_mse_summary, index=['SFR'])])
# sd
sim3_sd_results = pd.concat([pd.DataFrame(ols_sd_summary, index=['OLS']), pd.DataFrame(huber_sd_summary, index=['HUBER']),
                              pd.DataFrame(ransac_sd_summary, index=['RANSAC']), pd.DataFrame(sfr_sd_summary, index=['SFR'])])
# jb
sim3_jb_results = pd.concat([pd.DataFrame(ols_jb_summary, index=['OLS']), pd.DataFrame(huber_jb_summary, index=['HUBER']),
                              pd.DataFrame(ransac_jb_summary, index=['RANSAC']), pd.DataFrame(sfr_jb_summary, index=['SFR'])])

In [ ]:
# get latex
sim3_all_results = pd.concat([sim3_mse_results, sim3_bias_results, sim3_sd_results, sim3_jb_results])
print(sim3_all_results.to_latex(caption='Simulation Results for Effect Estimation - Scenario 3: Densely Distributed Outliers with Leverage'))
# save
sim3_all_results.to_csv(path+'/results/sim3_results.csv', index=False)

### Scenario 4: Clusters of Densely Distributed Outliers with Leverage

In [ ]:
# simulate data
# true slope
true_slope = 1
data = dgp(beta=true_slope, outlier_share=0.05, leverage=True, symmetry=True, verbose=False, seed=0, n_obs=1000)

# graphical evidence
# specify model
model = sm.OLS(endog=data.y, exog=pd.DataFrame(sm.add_constant(data.x0)))

# fit ols
ols_fit = model.fit()
ols_pred = ols_fit.fittedvalues
ols_weights = np.ones(len(data.y))

# fit huber
huber_fit = sm.RLM(endog=data.y, exog=pd.DataFrame(sm.add_constant(data.x0)), M=sm.robust.norms.HuberT()).fit()
huber_pred = huber_fit.fittedvalues
huber_weights = huber_fit.weights

# fit ransac
ransac_fit = RANSACRegressor(random_state=0)
ransac_fit.fit(X=pd.DataFrame(data.x0), y=data.y)
ransac_pred = pd.Series(ransac_fit.predict(X=pd.DataFrame(data.x0)))
ransac_weights = ransac_fit.inlier_mask_*1

# fit sfr
sfr = sf.SFR(linear_model=model, random_state=0)
sfr_fit = sfr.fit()
sfr_pred = pd.Series(sfr_fit.fittedvalues)
sfr_scores = sfr.score()
sfr_weights = sfr_fit.weights

In [ ]:
# save results for later plots
data_sim4 = data.copy()
scores_sim4 = sfr_scores.scores.copy()

#### Annealing

In [ ]:
sfr_annealing = sfr.anneal(share=0.1, n_boot=1000)
sfr_annealing.plot(xname='x0', title='Scenario 4: Clusters of Densely Distributed Outliers with Leverage', ylim=[0.6,1.2], dpi=300,
                   path=path+'/figures', fname='sim4_annealing.png')

In [ ]:
# format
sfr_annealing.figures['x0'][1].axhline(y=1, color='grey', linestyle='--', xmin=0, xmax=1.5, label="True")
labels = ["Estimate", "True", "95% CI"]
sfr_annealing.figures['x0'][1].get_legend().remove()
handles, _ = sfr_annealing.figures['x0'][1].get_legend_handles_labels()
# Slice list to remove first handle
sfr_annealing.figures['x0'][0].legend(handles = handles, labels = labels, loc = 'upper left', bbox_to_anchor=(0.125, 0.88))
sfr_annealing.figures['x0'][0]

In [ ]:
# save
sfr_annealing.figures['x0'][0].savefig(path+'/figures/sim4_annealing.png')

#### Scoring

In [ ]:
# plot the scores
sfr_scores.plot(xname='x0', xlabel='X', yname='Y', figsize=(10, 5), ylim=[-7.5,7.5], xlim=[-5,5], title='Scenario 4: Clusters of Densely Distributed Outliers with Leverage',
                dpi=300, path=path+'/figures', fname='sim4_scores.png')

#### Fitting

In [ ]:
# evaluate slope estimate

# bias
ols_bias_summary = {}
huber_bias_summary = {}
ransac_bias_summary = {}
sfr_bias_summary = {}

# mse
ols_mse_summary = {}
huber_mse_summary = {}
ransac_mse_summary = {}
sfr_mse_summary = {}

# SD
ols_sd_summary = {}
huber_sd_summary = {}
ransac_sd_summary = {}
sfr_sd_summary = {}

# JB
ols_jb_summary = {}
huber_jb_summary = {}
ransac_jb_summary = {}
sfr_jb_summary = {}

# true slope
true_slope = 1
sim_reps = 1000

# settings
obs_range = [100, 500, 1000]
outlier_share_range = [0.01, 0.025, 0.05]

# run simulation
for obs_idx in obs_range:
    for outlier_share_idx in outlier_share_range:
        
        # storage
        # bias
        ols_bias = []
        huber_bias = []
        ransac_bias = []
        sfr_bias = []
        
        # mse
        ols_mse = []
        huber_mse = []
        ransac_mse = []
        sfr_mse = []

        # storage for SD and JB
        ols_coef = []
        huber_coef = []
        ransac_coef = []
        sfr_coef = []
        
        for sim_idx in range(sim_reps):
            
            # generate data
            data = dgp(beta=true_slope, n_obs=obs_idx, outlier_share=outlier_share_idx, leverage=True, symmetry=True, verbose=False, seed=sim_idx)

            # specify model
            model = sm.OLS(endog=data.y, exog=pd.DataFrame(sm.add_constant(data.x0)))

            # fit ols
            ols_fit = model.fit()
            ols_bias.append(np.abs(true_slope - ols_fit.params[1]))
            ols_mse.append((true_slope - ols_fit.params[1]) ** 2)
            ols_coef.append(ols_fit.params[1])

            # fit huber
            np.random.seed(sim_idx)
            huber_fit = sm.RLM(endog=data.y, exog=pd.DataFrame(sm.add_constant(data.x0)), M=sm.robust.norms.HuberT()).fit()
            huber_bias.append(np.abs(true_slope - huber_fit.params[1]))
            huber_mse.append((true_slope - huber_fit.params[1]) ** 2)
            huber_coef.append(huber_fit.params[1])

            # fit ransac
            ransac_fit = RANSACRegressor(random_state=sim_idx)
            ransac_fit.fit(X=pd.DataFrame(data.x0), y=data.y)
            ransac_bias.append(np.abs(true_slope - ransac_fit.estimator_.coef_[0]))
            ransac_mse.append((true_slope - ransac_fit.estimator_.coef_[0]) ** 2)
            ransac_coef.append(ransac_fit.estimator_.coef_[0])

            # fit sfr
            sfr_fit = sf.SFR(linear_model=model, random_state=sim_idx).fit()
            sfr_bias.append(np.abs(true_slope - sfr_fit.params[1]))
            sfr_mse.append((true_slope - sfr_fit.params[1]) ** 2)
            sfr_coef.append(sfr_fit.params[1])
        
        # summarize resurls
        # ols
        ols_bias_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(ols_bias)
        ols_mse_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(ols_mse)
        ols_sd_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.std(ols_coef)
        ols_jb_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = stats.jarque_bera(ols_coef).pvalue
        
        # huber
        huber_bias_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(huber_bias)
        huber_mse_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(huber_mse)
        huber_sd_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.std(huber_coef)
        huber_jb_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = stats.jarque_bera(huber_coef).pvalue
        
        # ransac
        ransac_bias_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(ransac_bias)
        ransac_mse_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(ransac_mse)
        ransac_sd_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.std(ransac_coef)
        ransac_jb_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = stats.jarque_bera(ransac_coef).pvalue
        
        # sfr
        sfr_bias_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(sfr_bias)
        sfr_mse_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.mean(sfr_mse)
        sfr_sd_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = np.std(sfr_coef)
        sfr_jb_summary[str(obs_idx) + '_' + str(outlier_share_idx)] = stats.jarque_bera(sfr_coef).pvalue

In [ ]:
# summarize results
# bias
sim4_bias_results = pd.concat([pd.DataFrame(ols_bias_summary, index=['OLS']), pd.DataFrame(huber_bias_summary, index=['HUBER']),
                               pd.DataFrame(ransac_bias_summary, index=['RANSAC']), pd.DataFrame(sfr_bias_summary, index=['SFR'])])
# mse
sim4_mse_results = pd.concat([pd.DataFrame(ols_mse_summary, index=['OLS']), pd.DataFrame(huber_mse_summary, index=['HUBER']),
                              pd.DataFrame(ransac_mse_summary, index=['RANSAC']), pd.DataFrame(sfr_mse_summary, index=['SFR'])])
# sd
sim4_sd_results = pd.concat([pd.DataFrame(ols_sd_summary, index=['OLS']), pd.DataFrame(huber_sd_summary, index=['HUBER']),
                              pd.DataFrame(ransac_sd_summary, index=['RANSAC']), pd.DataFrame(sfr_sd_summary, index=['SFR'])])
# jb
sim4_jb_results = pd.concat([pd.DataFrame(ols_jb_summary, index=['OLS']), pd.DataFrame(huber_jb_summary, index=['HUBER']),
                              pd.DataFrame(ransac_jb_summary, index=['RANSAC']), pd.DataFrame(sfr_jb_summary, index=['SFR'])])

In [ ]:
# get latex
sim4_all_results = pd.concat([sim4_mse_results, sim4_bias_results, sim4_sd_results, sim4_jb_results])
print(sim4_all_results.to_latex(caption='Simulation Results for Effect Estimation - Scenario 4: Clusters of Densely Distributed Outliers with Leverage'))
# save
sim4_all_results.to_csv(path+'/results/sim4_results.csv', index=False)

### Summary

In [ ]:
# plot all scores together
fig, ([ax1, ax2], [ax3, ax4]) = plt.subplots(nrows=2, ncols=2, figsize=(20,10), dpi=300) # define the plot layout
fig.subplots_adjust(hspace=0.35)
# sim1 data
sim1_plot = ax1.scatter(x=data_sim1.x0, y=data_sim1.y, c=scores_sim1, cmap='RdYlGn', s=30)
ax1.set_ylim([-7.5,7.5])
ax1.set_xlim([-5,5])
ax1.title.set_text("Scenario 1: No Outliers")
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
# sim2 data
sim2_plot = ax2.scatter(x=data_sim2.x0, y=data_sim2.y, c=scores_sim2, cmap='RdYlGn', s=30)
ax2.set_ylim([-7.5,7.5])
ax2.set_xlim([-5,5])
ax2.title.set_text("Scenario 2: Randomly Distributed Outliers")
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
# sim2 data
sim3_plot = ax3.scatter(x=data_sim3.x0, y=data_sim3.y, c=scores_sim3, cmap='RdYlGn', s=30)
ax3.set_ylim([-7.5,7.5])
ax3.set_xlim([-5,5])
ax3.title.set_text("Scenario 3: Densely Distributed Outliers with Leverage")
ax3.set_xlabel('X')
ax3.set_ylabel('Y')
# sim2 data
sim4_plot = ax4.scatter(x=data_sim4.x0, y=data_sim4.y, c=scores_sim4, cmap='RdYlGn', s=30)
ax4.set_ylim([-7.5,7.5])
ax4.set_xlim([-5,5])
ax4.title.set_text("Scenario 4: Clusters of Densely Distributed Outliers with Leverage")
ax4.set_xlabel('X')
ax4.set_ylabel('Y')

# add legend
legend = ax4.legend(*sim4_plot.legend_elements(), title="Reliability Score",
                    bbox_to_anchor=(-1.2, -0.4, 2.2, .102), loc=3,
                    ncol=12, mode="expand", borderaxespad=0., fancybox=True, shadow=True)
ax4.add_artist(legend)
plt.show()
fig.savefig(path+'/figures/scores_sim_all.png', bbox_inches='tight')

In [ ]:
# all mse results
all_mse_results = pd.concat([sim1_mse_results.loc[['OLS', 'SFR'], ['1000_0.0']], sim2_mse_results.loc[['OLS', 'SFR'], ['1000_0.05']],
                             sim3_mse_results.loc[['OLS', 'SFR'], ['1000_0.05']], sim4_mse_results.loc[['OLS', 'SFR'], ['1000_0.05']]], axis=1)
# all abs bias results
all_bias_results = pd.concat([sim1_bias_results.loc[['OLS', 'SFR'], ['1000_0.0']], sim2_bias_results.loc[['OLS', 'SFR'], ['1000_0.05']],
                              sim3_bias_results.loc[['OLS', 'SFR'], ['1000_0.05']], sim4_bias_results.loc[['OLS', 'SFR'], ['1000_0.05']]], axis=1)
# all sdev results
all_sd_results = pd.concat([sim1_sd_results.loc[['OLS', 'SFR'], ['1000_0.0']], sim2_sd_results.loc[['OLS', 'SFR'], ['1000_0.05']],
                            sim3_sd_results.loc[['OLS', 'SFR'], ['1000_0.05']], sim4_sd_results.loc[['OLS', 'SFR'], ['1000_0.05']]], axis=1)
# all jberra results
all_jb_results = pd.concat([sim1_jb_results.loc[['OLS', 'SFR'], ['1000_0.0']], sim2_jb_results.loc[['OLS', 'SFR'], ['1000_0.05']],
                            sim3_jb_results.loc[['OLS', 'SFR'], ['1000_0.05']], sim4_jb_results.loc[['OLS', 'SFR'], ['1000_0.05']]], axis=1)

In [ ]:
all_sim_results = pd.concat([all_mse_results, all_bias_results, all_sd_results, all_jb_results])
print(all_sim_results.to_latex(caption='Fitting: Comparison of Simulation Scenarios'))
# save
all_sim_results.to_csv(path+'/results/all_sim_results.csv', index=False)